In [1]:
# coding: utf-8
import sys
import pandas as pd
import numpy as np
import os 
import time
import json
import requests
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
from bs4 import BeautifulSoup

board_list = [
    'NBA', 'Stock', 'Lifeismoney',
    'Baseball', 'movie', 'LoL', 'car', 
    'MobileComm', 'marvel', 'PC_Shopping', 'Tech_Job', 'Salary', 
    'StupidClown', 'CVS', 'Soft_Job', 'Bank_Service', 'Finance'
]

# 特殊符號分割
pattern = r'\n| |\*|？|　|：|,|\.|/|;|\'|`|\[|\]|<|>|\?|:|"|\{|\}|\~|!|@|#|\$|%|\^|&|\(|\)|-|=|\_|\+|，|。|、|；|‘|’|【|】|·|！| |…|（|）|〔|／|〕|「|」'

user_dict = {}
crawl_pages = 10

In [2]:
def getSeg(text):
    if not text:
        return ''
    if len(text) == 1:
        return text
    if text in word_dict:
        return text
    else:
        small = len(text) - 1
        text = text[0:small]
        return getSeg(text)

def ptt_crawler(n_page):
    if not os.path.exists('source/ptt/'):
        os.mkdir('source/ptt/')
    target_content = u'※ 發信站: 批踢踢實業坊(ptt.cc),'
    for bb in board_list:
        url = "https://www.ptt.cc/bbs/" + bb + "/index.html"
        file_name = "source/ptt/" + bb + ".txt"
        print('-'*20, bb, '-'*20)
        
        for i in range(n_page): 
            r = requests.get(url, verify=False)
            r.encoding = 'utf-8'
            soup = BeautifulSoup(r.text,"html.parser")
            sel = soup.select("div.title a") #標題
            u = soup.select("div.btn-group.btn-group-paging a") #a標籤
            print('Starting crawler',url)
            for s in sel: 
                text_url = 'https://www.ptt.cc' + s["href"]
                response = requests.get(text_url, verify=False)
                response.encoding = 'utf-8'
                soup = BeautifulSoup(response.text, 'lxml')
                try:
                    content = soup.find(id="main-content").text
                    content = content.replace('\n', '').split('標題')[1].split(target_content)[0].replace('--', '')     
                    push_content = soup.find_all("span", {"class": "f3 push-content"})
                    with open(file_name, "a") as text_file:
                        try:
                            text_file.write(content)
                            for x in [x.text.replace(':', '') for x in push_content]:
                                text_file.write(x)
                            text_file.close()
                        except:
                            pass  
                except:
                    continue
            # 如果本頁是第一頁，則跳出迴圈。
            if 'index1.html' in url:
                break
            else:
                # url更新為上一頁網址
                url = "https://www.ptt.cc"+ u[1]["href"] 
                time.sleep(10)

<hr>
PTT爬蟲<br>
預設爬10頁<br>
如果進頁面需要確認滿18歲則會Error<br>
後續再調整:O<br>

In [3]:
ptt_crawler(crawl_pages)

-------------------- NBA --------------------
Starting crawler https://www.ptt.cc/bbs/NBA/index.html
-------------------- Stock --------------------
Starting crawler https://www.ptt.cc/bbs/Stock/index.html
-------------------- Lifeismoney --------------------
Starting crawler https://www.ptt.cc/bbs/Lifeismoney/index.html
-------------------- Baseball --------------------
Starting crawler https://www.ptt.cc/bbs/Baseball/index.html
-------------------- movie --------------------
Starting crawler https://www.ptt.cc/bbs/movie/index.html
-------------------- LoL --------------------
Starting crawler https://www.ptt.cc/bbs/LoL/index.html
-------------------- car --------------------
Starting crawler https://www.ptt.cc/bbs/car/index.html
-------------------- MobileComm --------------------
Starting crawler https://www.ptt.cc/bbs/MobileComm/index.html
-------------------- marvel --------------------
Starting crawler https://www.ptt.cc/bbs/marvel/index.html
-------------------- PC_Shopping ----

In [4]:
source_dir = os.listdir('source')
for src_cate in source_dir:
    filedir = ['source/' + src_cate + '/' + x for x in os.listdir('source/'+src_cate)]

print(source_dir)
print(filedir)

['ptt']
['source/ptt/Bank_Service.txt', 'source/ptt/Baseball.txt', 'source/ptt/car.txt', 'source/ptt/CVS.txt', 'source/ptt/Finance.txt', 'source/ptt/Lifeismoney.txt', 'source/ptt/LoL.txt', 'source/ptt/marvel.txt', 'source/ptt/MobileComm.txt', 'source/ptt/movie.txt', 'source/ptt/NBA.txt', 'source/ptt/PC_Shopping.txt', 'source/ptt/Salary.txt', 'source/ptt/Soft_Job.txt', 'source/ptt/Stock.txt', 'source/ptt/StupidClown.txt', 'source/ptt/Tech_Job.txt']


In [5]:
for fn in filedir:
    print(fn, '-'*20)
    ff = open(fn, 'r').readlines()
    for string_raw in ff:
        result_list = re.split(pattern, string_raw)
        for string in result_list:
            string = string.replace('\n', '')
            for n_ in list(range(len(string))):
                for n_len in list(range(n_, len(string))):
                    nn = string[n_:n_len+1]
                    if nn in user_dict.keys():
                        user_dict[nn] += 1
                    else:
                        user_dict[nn] = 1


source/ptt/Bank_Service.txt --------------------
source/ptt/Baseball.txt --------------------
source/ptt/car.txt --------------------
source/ptt/CVS.txt --------------------
source/ptt/Finance.txt --------------------
source/ptt/Lifeismoney.txt --------------------
source/ptt/LoL.txt --------------------
source/ptt/marvel.txt --------------------
source/ptt/MobileComm.txt --------------------
source/ptt/movie.txt --------------------
source/ptt/NBA.txt --------------------
source/ptt/PC_Shopping.txt --------------------
source/ptt/Salary.txt --------------------
source/ptt/Soft_Job.txt --------------------
source/ptt/Stock.txt --------------------
source/ptt/StupidClown.txt --------------------
source/ptt/Tech_Job.txt --------------------


In [6]:
# Save dictionary
json_text = json.dumps(user_dict)
f = open("dict.json","w")
f.write(json_text)
f.close()


In [7]:
# 整理後存成txt
pd_dict = pd.DataFrame.from_dict(user_dict, orient='index')
pd_dict.reset_index(inplace=True)

pd_dict.columns = ['word', 'count']
pd_dict = pd_dict[pd_dict['count'] > 10]
pd_dict = pd_dict[pd_dict['word'].str.len() > 1]
pd_dict = pd_dict[pd_dict['word'].str.len() < 9]
pd_dict['count_log'] = np.log(pd_dict['count'])
pd_dict['word_len'] = pd_dict['word'].str.len()
pd_dict['count_multiply_word_len'] = pd_dict['count'] * pd_dict['word_len']
pd_dict['count_multiply_word_len_square'] = pd_dict['count'] * pd_dict['word_len'] * pd_dict['word_len']
pd_dict['count_log_multiply_word_len'] = pd_dict['count_log'] * pd_dict['word_len']
pd_dict['count_log_multiply_word_len_square'] = pd_dict['count_log'] * pd_dict['word_len'] * pd_dict['word_len']

pd_dict.to_csv('word.txt', index=False)